# PyCUDA installation


In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 16.3MB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
     |████████████████████████████████| 81kB 13.3MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4537514 sha256=2a3b8a102148d8de901cabac804674b5ed1be89bcded0d93ffe7ffe2ad3ef2b7
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=ab844ce2ae06eee1c59f06d97f693778d02ace89d399b315c10ed4097a513068
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c9800d6d0ebc2a554cb89829cc24
Successfully built pycuda pytools




---



# Version #1: using ```SourceModule```

PyCUDA initialization

In [0]:
import numpy as np

# --- PyCUDA initialization
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


iDivUp function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` ```+1```.



In [0]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

In [0]:
########
# MAIN #
########

Defining two CUDA events that will be used to measure execution time.

In [0]:
start = cuda.Event()
end   = cuda.Event()

Number of array elements

In [0]:
N = 100000

Number of threads per block

In [0]:
BLOCKSIZE = 256

Create two host vectors ```h_a``` and ```h_b``` of ```N``` random entries. ```np.random.randn``` returns ```float64```'s.

In [0]:
h_a = np.random.randn(1, N)
h_b = np.random.randn(1, N)

Cast ```h_a``` and ```h_b``` to single precision (```float32```). 

In [0]:
h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)

Allocate ```h_a.nbytes```, ```h_b.nbytes``` and ```h_c.nbytes``` of GPU device memory space pointed to by ```d_a```, ```d_b``` and ```d_c```, respectively.

In [0]:
d_a = cuda.mem_alloc(h_a.nbytes)
d_b = cuda.mem_alloc(h_b.nbytes)
d_c = cuda.mem_alloc(h_a.nbytes)

Copy the ```h_a``` and ```h_b``` arrays from host to the device arrays ```d_a``` and ```d_b```, respectively.

In [0]:
cuda.memcpy_htod(d_a, h_a)
cuda.memcpy_htod(d_b, h_b)

Define the CUDA kernel function ```deviceAdd``` as a string. ```deviceAdd``` performs the elementwise summation of ```d_a``` and ```d_b``` and puts the result in ```d_c```.



In [0]:
mod = SourceModule("""
  #include <stdio.h>
  __global__ void deviceAdd(float * __restrict__ d_c, const float * __restrict__ d_a, const float * __restrict__ d_b, const int N)
  {
    const int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    d_c[tid] = d_a[tid] + d_b[tid];
  } 
  """)

Define a reference to the ```__global__``` function ```deviceAdd```.

In [0]:
deviceAdd = mod.get_function("deviceAdd")

Define the block size ```blockDim``` and the grid size ```gridDim```.

In [0]:
blockDim  = (BLOCKSIZE, 1, 1)
gridDim   = (int(iDivUp(N, BLOCKSIZE)), 1, 1)

Invoke the ```deviceAdd``` function. 
Note that, up to here, ```N``` is an *object* of ```class int``` and not an integer number. Therefore, before using it, we must cast it to ```np.int32``` which is essentially the standard, single precision, floating point type.
Before launching ```deviceAdd```, the ```start``` and ```end``` events are recorded, so that the execution time can be measured.
Note that, before the processing time can be measured, all the activities in the current context must be ceased. This is the reason why ```end.synchronize()``` is invoked. Remember that the host and device executions are asynchronous. Furthermore, with the event record, the device will record a time stamp for the event when it reaches that event in the stream. Without synchronization, it happens that the ```end``` event is recorded after the ```deviceAdd``` function execution is actually terminated, as we expect, but the ```print``` function is executed before ```deviceAdd``` has actually finished its execution.


In [15]:
# --- Warmup execution
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)

start.record()
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)
end.record() 
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

Processing time = 0.000517s


Allocate host space and copy results from device to host.

In [0]:
h_c = np.empty_like(h_a)
cuda.memcpy_dtoh(h_c, d_c)

Check if the device processing results are as expected.

In [17]:
if np.array_equal(h_c, h_a + h_b):
  print("Test passed!")
else :
  print("Error!")

Test passed!


Finally, flush context printf buffer. Without flushing, no printout may be returned.

In [0]:
cuda.Context.synchronize()